<a href="https://colab.research.google.com/github/thinkaboutcode/cheatsheets/blob/main/spark_cheatsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Data Frames

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=2e85ed482e5cb9352e0103ed2661f9dfc2156257cc9e75c5d1ad92d1bba83410
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [30]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySpark DataFrames Tutorial") \
    .getOrCreate()


In [31]:
spark

## Creating a DataFrame from a List

In [6]:
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Id"])
df.show()


+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



## Creating a DataFrame from a Dictionary

In [17]:
data_dict = [{"Name": "Alice", "Id": 1}, {"Name": "Bob", "Id": 2}, {"Name": "Cathy", "Id": 3}]
df_dict = spark.createDataFrame(data_dict)

df_dict.show()


+---+-----+
| Id| Name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Cathy|
+---+-----+



In [18]:
df.show()

+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



In [20]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Id: long (nullable = true)



## Selecting Columns

In [21]:
df.select("Name").show()

+-----+
| Name|
+-----+
|Alice|
|  Bob|
|Cathy|
+-----+



In [19]:
df.take(2)

[Row(Name='Alice', Id=1), Row(Name='Bob', Id=2)]

In [14]:
df.collect()

[Row(Name='Alice', Id=1), Row(Name='Bob', Id=2), Row(Name='Cathy', Id=3)]

## Filtering Data

In [22]:
df.filter(df.Id > 1).show()

+-----+---+
| Name| Id|
+-----+---+
|  Bob|  2|
|Cathy|  3|
+-----+---+



## Adding a New Column

In [24]:
from pyspark.sql.functions import col

df = df.withColumn("NewId", col("Id") + 10)
df.show()

+-----+---+-----+
| Name| Id|NewId|
+-----+---+-----+
|Alice|  1|   11|
|  Bob|  2|   12|
|Cathy|  3|   13|
+-----+---+-----+



## Renaming a Column

In [25]:
df = df.withColumnRenamed("NewId", "UpdatedId")
df.show()

+-----+---+---------+
| Name| Id|UpdatedId|
+-----+---+---------+
|Alice|  1|       11|
|  Bob|  2|       12|
|Cathy|  3|       13|
+-----+---+---------+



## Dropping a Column

In [26]:
df = df.drop("UpdatedId")
df.show()

+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



## Aggregations

In [27]:
from pyspark.sql import functions as F

df.agg(F.avg("Id")).show()

+-------+
|avg(Id)|
+-------+
|    2.0|
+-------+



In [28]:
df_grouped = df.groupBy("Name").count()
df_grouped.show()

+-----+-----+
| Name|count|
+-----+-----+
|Alice|    1|
|  Bob|    1|
|Cathy|    1|
+-----+-----+



## Saving DataFrame

In [ ]:
df.write.csv("output.csv", header=True)

## Stopping the Spark Session

In [ ]:
spark.stop()

# Spark SQL

In [32]:
df.createOrReplaceTempView("people")

## Selecting Data

In [33]:
result = spark.sql("SELECT * FROM people")
result.show()

+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



## Filtering Data

In [34]:
filtered_result = spark.sql("SELECT * FROM people WHERE Id > 1")
filtered_result.show()

+-----+---+
| Name| Id|
+-----+---+
|  Bob|  2|
|Cathy|  3|
+-----+---+



## Aggregating Data

In [35]:
agg_result = spark.sql("SELECT COUNT(*) as count FROM people")
agg_result.show()

+-----+
|count|
+-----+
|    3|
+-----+



In [36]:
grouped_result = spark.sql("SELECT Name, COUNT(*) as count FROM people GROUP BY Name")
grouped_result.show()

+-----+-----+
| Name|count|
+-----+-----+
|Alice|    1|
|  Bob|    1|
|Cathy|    1|
+-----+-----+



## Sorting Data

In [37]:
sorted_result = spark.sql("SELECT * FROM people ORDER BY Id DESC")
sorted_result.show()

+-----+---+
| Name| Id|
+-----+---+
|Cathy|  3|
|  Bob|  2|
|Alice|  1|
+-----+---+



## Creating and Using Temporary Views

In [39]:
data2 = [("David", 4), ("Eva", 5)]
columns = ["Name", "Id"]
df2 = spark.createDataFrame(data2, schema=columns)
df2.createOrReplaceTempView("more_people")

## Querying from both views

In [40]:
combined_result = spark.sql("""
    SELECT Name, Id FROM people
    UNION ALL
    SELECT Name, Id FROM more_people
""")
combined_result.show()

+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
|David|  4|
|  Eva|  5|
+-----+---+



# Saving Query Results

In [ ]:
agg_result.write.csv("agg_result.csv", header=True)